<a href="https://colab.research.google.com/github/YaCpotato/python/blob/master/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

やること  
* XGBoost理論調査
* LightGBM CatBoostとの比較

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd drive/'My Drive'/'Colab Notebooks'/

In [0]:
import pandas as pd
import io
train = pd.read_csv('./poker-hand-training-true.csv', header=None)
test = pd.read_csv('./poker-hand-testing.csv', header=None)
train.head()
test.head()

こういうデータに対して(どういうデータか説明もできない)するべき前処理が未だうかばない筆者です。アドバイス求む。  
PandasのDataFrameも使おうとしたがまだ機械学習のいちプロセスで使い通せない段階

# データを整形する
* ラベルデータの取り出し(正確には代入してから削除してる)
* 学習データ、テストデータの分割

In [0]:
from sklearn import preprocessing
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize

train_Y = train[:][10]
test_Y = test[:][10]

X_train, X_test, = train_test_split(train,train_size=0.7)
Y_train,Y_test = train_test_split(train_Y,train_size=0.7)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

X_train.drop(10,axis=1,inplace=True)
X_test.drop(10,axis=1,inplace=True)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(17507, 11) (17507,) (7503, 11) (7503,)
(17507, 10) (17507,) (7503, 10) (7503,)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [0]:
clf = xgb.XGBClassifier()

@param max_depth: 決定木の葉の数

In [0]:
clf_cv = GridSearchCV(clf, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)

In [0]:
%%time
clf_cv.fit(X_train, Y_train,verbose=1)
# 改めて最適パラメータで学習
clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(X_train, Y_train,verbose=1)

(17507, 10) (17507,)
Fitting 3 folds for each of 9 candidates, totalling 27 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [2, 4, 6],
                         'n_estimators': [50, 100, 200]},
             pre_dispatch='2*n_jobs', refit=True, retur

In [0]:
# 学習モデルの評価
from sklearn.metrics import confusion_matrix,classification_report
pred = clf.predict(X_test)
print (confusion_matrix(Y_test, pred))
print (classification_report(Y_test, pred))

[[3221  591    1    0    0    0    0    0    0]
 [1147 1948   12    2    1    0    0    0    0]
 [  28  315   11    1    0    0    0    0    0]
 [   5  145    1    9    0    0    0    0    0]
 [   2   29    0    0    0    0    0    0    0]
 [  16    1    0    0    0    1    0    0    0]
 [   0    9    0    1    0    0    0    0    0]
 [   0    3    0    0    0    0    0    0    0]
 [   1    1    0    0    0    1    0    0    0]]
              precision    recall  f1-score   support

           0       0.73      0.84      0.78      3813
           1       0.64      0.63      0.63      3110
           2       0.44      0.03      0.06       355
           3       0.69      0.06      0.10       160
           4       0.00      0.00      0.00        31
           5       0.50      0.06      0.10        18
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00         3
           9       0.00      0.00      0.00         3

    accuracy             

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
